<a href="https://colab.research.google.com/github/thewildchip/crispr-genie-lab/blob/main/off_target/00_data_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0) Data Loading for Off Target Model

In [ ]:
from pathlib import Path
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from mypackage.utils import  *
from mypackage.config import *

In [ ]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/dagrate/public_data_crisprCas9/refs/heads/main/data/kleinstiver_5gRNA_wholeDataset.csv")
df = df.dropna()
X = df["sqRNA_seq", "off_seq", "Read"]
y = df["label"]

def onehot_encode_sequences(df: pd.DataFrame, seq_col="23-nt_sequence"):
    sequences = df[seq_col].values
    n_sequences = len(sequences)
    seq_length = len(sequences[0])
    bases = ["A", "C", "G", "T"]

    # Create an empty array: shape (num_sequences, seq_length, 4)
    onehot = np.zeros((n_sequences, seq_length, 4), dtype=int)

    # Fill in the one-hot array
    for i, base in enumerate(bases):
        onehot[:, :, i] = (np.array([list(seq) for seq in sequences]) == base)

    # Flatten to columns
    col_names = [f"pos_{pos}_{base}" for pos in range(seq_length) for base in bases]
    onehot_flat = onehot.reshape(n_sequences, seq_length * 4)

    # Return a new DataFrame with one-hot columns
    #return pd.DataFrame(onehot_flat, columns=col_names, index=df.index).merge(df.drop(columns=[seq_col]), left_index=True, right_index=True)
    return df.drop(columns=[seq_col]).merge(pd.DataFrame(onehot_flat, columns=col_names, index=df.index), left_index=True, right_index=True)


X_rna = df ["sqRNA_seq"],apply(onehot_encode_sequences)
X_rna_np = np.array(X_rna.tolist())

X_off = df ["off_seq"],apply(onehot_encode_sequences)
X_off_np = np.array(X_off.tolist())

df.fit(X,y)
df.head()
train_X, train_y, val_X, val_y, X_test, y_test= train_test_split(X, y, test_size=0.2, random_state=42) 
print(df.predict())


reg = DecisionTreeRegressor(
    max_depth=5,         # prevents overfitting
    min_samples_leaf=10, # ensures leaves are not too small
    random_state=42
)

# Train
reg.fit(train_X, train_y)

# Predict
y_pred = reg.predict(X_test)

# Evaluate
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R2:", r2) 

